In [30]:
import anthropic

In [31]:
from dotenv import load_dotenv

In [32]:
load_dotenv()

True

In [33]:
client_anthropic = anthropic.Anthropic(api_key = os.getenv('CLAUDE_API_KEY'))

In [34]:
import os

In [35]:
import json

In [36]:
from supabase import create_client, Client

In [37]:
supabase_api_key = os.environ.get("SUPBASE_KEY")
supabase_url = os.environ.get("SUPABASE_URL")

In [38]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [12]:
researched_data = [{'event_id': '299_2026-02-08',
  'news_date': '2026-02-08',
  'output': '- Verified: The earbuds rumor centers on an internal code name for OpenAI’s first consumer hardware device being Sweetpea. Multiple credible outlets report this codename as part of the leak cycle about the device. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: Several credible outlets note that the project is not officially confirmed and remains a leak/rumor at this time. TechCrunch explicitly describes the details as coming from leaks and “tipsters,” not an official OpenAI confirmation; Axios likewise characterizes the information as rumors shared at Davos. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: The same reporting that mentions Sweetpea also references a potential second consumer device codenamed Gumdrop in the rumor mill. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Added confirmed hardware context: OpenAI disclosed at Davos that it is building a new AI device, though details remain scarce. Timeline given by OpenAI officials in these discussions points to a likely release in the second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Confirmed: OpenAI’s prior hardware efforts are linked to its 2025 acquisition of Jony Ive’s design firm io (the deal was reported to be around $6.5 billion). Ive is associated with leading the hardware design work for the product family. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: OpenAI’s hardware plans discussed publicly have included a screen-free, pocketable device concept as part of the early product narrative. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: Reported design descriptions for the Sweetpea concept include a main unit described as metal and egg-shaped, with two pill-shaped modules that would sit behind the ear. These details appear in multiple outlets drawing on the same leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (design details corroborated by multiple outlets): The rumored hardware would reportedly use a 2-nanometer-class processor (with Exynos as a favored reference) for on-device AI tasks, aligned with the “phone-like” compute ambitions discussed in the leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (manufacturing partner context): Reports mention Foxconn as a manufacturing partner for the OpenAI devices, with some mention of Luxshare as a possibility; this is described in the leakage coverage and echoed by outlets citing Asian reporting. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: First-year shipment targets cited in the leaks are in the 40–50 million units range, with September 2026 as a possible launch window in at least some outlets. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed (scope/roadmap detail in leaks): Some leakage coverage claims OpenAI is planning a broader hardware roadmap (up to five devices by 2028), with Sweetpea at the front of that lineup. This detail appears in multiple leak-focused outlets. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n\n- Not corroborated by credible outlets (branding name caveat): The claim that the consumer-facing name will be “Dime” lacks confirmation from established major outlets. Some blogs and tech sites have floated “Dime” as branding based on patent filings or rumor chatter, but there is no widely credited report in outlets like TechCrunch, Axios, or The Indian Express confirming “Dime.” Examples of circulating but less-established sources include Gizmochina and other aggregators. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\n- Note on the current status of confirmation: The available reporting from January–February 2026 consistently describes the information as leaks/rumors rather than official confirmation; OpenAI executives acknowledged work on a hardware device but kept specifics (design, branding, features) under wraps. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nSources referenced above (examples of main reporting):\n- TechCrunch: OpenAI aims to ship its first device in 2026, and it could be earbuds (codename Sweetpea; design, chip, and unit targets discussed). ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n- Indian Express: OpenAI’s first product could be ChatGPT-powered earbuds; codename Sweetpea; design details and 2nm Samsung Exynos chip mentioned. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n- Android Authority: OpenAI earbuds leak with Sweetpea codename; design description; BOM/production cost hints; September launch target; five-device roadmap mentioned. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n- Axios: OpenAI’s secret device coming; Davos timeline; mentions Sweetpea and Gumdrop leaks; timeline for second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n- The Verge: OpenAI hardware prototype discussions tied to Jony Ive and the hardware program (context around the hardware push). ([theverge.com](https://www.theverge.com/news/827607/openai-hardware-prototype-chatgpt-jony-ive-sam-altman?utm_source=openai))\n- (Caveat on Dime branding) Gizmochina / other aggregators noting a claimed consumer name Dime; these are not widely corroborated by major outlets. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\nNotes:\n- All statements above about Sweetpea, the rumored features, and the general timeline are based on published reporting as of February 8, 2026; several items (notably the consumer-name “Dime”) are not yet verified by major, high-profile outlets and are reported only by a subset of non-primary sources. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nIf you’d like, I can add direct excerpts or align these points to a formal, source-by-source bibliography.'}]

In [49]:
result = supabase.table('research_assistant')\
    .select('id, event_id, news_date, output')\
.eq('news_date', '2026-02-17')\
    .order('created_at', desc=False)\
    .execute()

In [50]:
result.data

[{'id': 166,
  'event_id': '632_2026-02-17',
  'news_date': '2026-02-17',
  'output': '• WHAT IS NEW — Meta has begun integrating Manus AI into Ads Manager, giving advertisers an AI work partner to automate tasks such as data analysis and report generation directly within Ads Manager. ([socialmediatoday.com](https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/?utm_source=openai))\n\n• ACQUISITION BACKGROUND — Manus AI is an autonomous general-purpose AI agent; Meta announced its acquisition in December 2025 for a deal reported to be worth over $2 billion. ([techcrunch.com](https://techcrunch.com/2025/12/29/meta-just-bought-manus-an-ai-startup-everyone-has-been-talking-about/?utm_source=openai))\n\n• CHINA TIES SEVERED — Post-acquisition, Meta confirmed there will be no continuing Chinese ownership in Manus and that China-related operations will cease; Manus will relocate to Singapore and CEO Xiao Hong will join Meta. ([apnews.com](https://apne

In [41]:
final_summary_5 = []

In [29]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.

                TASK: Write a news article using ONLY the information provided by our research assistant.
                
                Follow this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                - Explain it in a simple way such that a non-tech person who has no background about this also understands its impact
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                ☑ There should be no em-dashes in the article
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:
                
                {i['output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_4.append(article_json)

        

        # save_article(article_json)
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [51]:
a = [{'id': 166,
  'event_id': '632_2026-02-17',
  'news_date': '2026-02-17',
  'output': '• WHAT IS NEW — Meta has begun integrating Manus AI into Ads Manager, giving advertisers an AI work partner to automate tasks such as data analysis and report generation directly within Ads Manager. ([socialmediatoday.com](https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/?utm_source=openai))\n\n• ACQUISITION BACKGROUND — Manus AI is an autonomous general-purpose AI agent; Meta announced its acquisition in December 2025 for a deal reported to be worth over $2 billion. ([techcrunch.com](https://techcrunch.com/2025/12/29/meta-just-bought-manus-an-ai-startup-everyone-has-been-talking-about/?utm_source=openai))\n\n• CHINA TIES SEVERED — Post-acquisition, Meta confirmed there will be no continuing Chinese ownership in Manus and that China-related operations will cease; Manus will relocate to Singapore and CEO Xiao Hong will join Meta. ([apnews.com](https://apnews.com/article/aaf01029923011a403ceeb949cf3db5e?utm_source=openai))\n\n• ADS MANAGER ROLLOUT — All Meta advertisers can access Manus within the Ads Manager Tools menu; the rollout appears incremental with in-app prompts and examples of usage shown to some users. ([socialmediatoday.com](https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/?utm_source=openai))\n\n• DEAL TERMS — Official financial terms were not disclosed; The Wall Street Journal has reported the transaction value at over $2 billion. ([apnews.com](https://apnews.com/article/aaf01029923011a403ceeb949cf3db5e?utm_source=openai))\n\n• MANUS CAPABILITIES — Manus supports autonomous multi-step tasks (e.g., report-building, market/audience research, data analysis, coding) and is intended to operate across Meta’s consumer and business products. ([techcrunch.com](https://techcrunch.com/2025/12/29/meta-just-bought-manus-an-ai-startup-everyone-has-been-talking-about/?utm_source=openai))\n\n• STRATEGIC RATIONALE — The integration represents Meta’s shift from conversational AI toward agentic infrastructure, embedding autonomous agents to execute tasks within Meta’s platforms. ([aipressroom.com](https://aipressroom.com/meta-acquires-manus/?utm_source=openai))\n\n• ROLLOUT SCOPE AND TIMING — The integration is being rolled out within Ads Manager and is intended to scale across Meta’s systems, with continued emphasis on embedding Manus as a workflow automation tool. ([socialmediatoday.com](https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/?utm_source=openai))\n\n• REGULATORY/GEOPOLITICAL CONTEXT — Manus’s origin (China-linked) drew attention; Meta has stressed that there is no ongoing Chinese ownership and emphasizes geographic data protections as part of the integration, amid broader scrutiny of Chinese investment in U.S. AI tech. ([techcrunch.com](https://techcrunch.com/2025/12/29/meta-just-bought-manus-an-ai-startup-everyone-has-been-talking-about/?utm_source=openai))\n\n• PRODUCT STATUS POST-ACQUISITION — Manus will continue operating as a standalone service while being integrated into Meta’s AI ecosystem (e.g., Meta AI) and across Meta’s consumer/business products; most Manus staff are joining Meta as part of the integration. ([apnews.com](https://apnews.com/article/aaf01029923011a403ceeb949cf3db5e?utm_source=openai))'},
 {'id': 167,
  'event_id': '643_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Unilever and Google Cloud announce a five-year partnership to accelerate Unilever’s digital transformation using Google Cloud’s AI, data capabilities, platform, and next-generation marketing tools (Vertex AI). ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- The alliance will unite Unilever’s global brand portfolio (including Dove, Vaseline, and Hellmann’s) with Google Cloud’s AI to advance cutting-edge technology and agentic commerce. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- Unilever will migrate its integrated data and cloud platform to Google Cloud to create an enterprise-wide, AI-first digital backbone to generate demand faster and turn data into actionable insights. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- The partnership will deploy Google Cloud’s Vertex AI as the enterprise AI platform to build capabilities in brand discovery, measurement, and AI-augmented marketing. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- Three core pillars define the collaboration: agentic commerce and marketing intelligence; an integrated data and cloud foundation; and advanced AI. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- The initiative aims to reshape how CP G brands are discovered and shopped, aligning with AI-driven, conversational consumer journeys. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- Willem Uijen, Unilever’s chief supply chain and operations officer, states that technology is moving to the core of value creation and AI-enabled capability will drive future growth. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- Tara Brady, Google Cloud president for Europe, the Middle East and Africa, notes the use of Gemini and the goal of a system of intelligence that reasons, learns, and acts to boost agility and consumer engagement. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))\n\n- The release describes agentic workflows as intelligent systems capable of executing complex tasks across Unilever’s business processes, enabled by the collaboration. ([prnewswire.com](https://www.prnewswire.com/news-releases/unilever-and-google-cloud-partner-to-pioneer-next-generation-of-consumer-goods-technologies-302688857.html))'},
 {'id': 168,
  'event_id': '644_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core development: Sarvam AI unveiled Sarvam Kaze, its first AI-powered smart glasses, demonstrated at the India AI Impact Summit 2026; Prime Minister Narendra Modi was shown testing the device on the expo floor. ([indiatoday.in](https://www.indiatoday.in/technology/news/story/sarvam-teases-new-ai-smartglasses-at-india-ai-impact-summit-pm-modi-gives-it-a-try-2869547-2026-02-17))\n\n- Design/manufacturing claim: The company says Kaze is designed and built in India, positioning it as one of the first India-made AI hardware devices. ([indiatoday.in](https://www.indiatoday.in/technology/news/story/sarvam-teases-new-ai-smartglasses-at-india-ai-impact-summit-pm-modi-gives-it-a-try-2869547-2026-02-17))\n\n- Launch plan: Sarvam Kaze is slated to go on sale in May 2026, with retail availability announced for the Indian market. ([indiatoday.in](https://www.indiatoday.in/technology/news/story/sarvam-teases-new-ai-smartglasses-at-india-ai-impact-summit-pm-modi-gives-it-a-try-2869547-2026-02-17))\n\n- Hardware features (teased): The glasses appear to include two front cameras and embedded microphones and speakers for voice interactions; front-camera cutouts and LED indicators were noted in previews. ([gadgets360.com](https://www.gadgets360.com/wearables/news/ai-impact-summit-2026-sarvam-kaze-ai-smart-glasses-showcased-launch-timeline-revealed-11031424))\n\n- Core function and UX: Sarvam Kaze is described as an “AI First” device that listens, understands, responds, and captures what the wearer sees, with the ability to build custom experiences via the Sarvam platform. ([gadgets360.com](https://www.gadgets360.com/wearables/news/ai-impact-summit-2026-sarvam-kaze-ai-smart-glasses-showcased-launch-timeline-revealed-11031424))\n\n- On-device AI and language focus: The glasses will run on Sarvam’s own foundational models trained for Indian languages and use cases, leveraging in-house AI for on-device capabilities. ([indiatoday.in](https://www.indiatoday.in/technology/news/story/sarvam-teases-new-ai-smartglasses-at-india-ai-impact-summit-pm-modi-gives-it-a-try-2869547-2026-02-17))\n\n- Developer ecosystem: The company indicated developers will be able to build custom apps and experiences on the Sarvam platform for Kaze. ([gadgets360.com](https://www.gadgets360.com/wearables/news/ai-impact-summit-2026-sarvam-kaze-ai-smart-glasses-showcased-launch-timeline-revealed-11031424))\n\n- Offline/on-device capability (Sarvam Edge context): Sarvam has emphasized an on-device/offline AI approach (Sarvam Edge) in its broader product strategy, aligning with privacy and low-latency use. ([business-standard.com](https://www.business-standard.com/technology/tech-news/india-ai-summit-2026-what-is-sarvam-kaze-ai-glasses-pm-modi-wore-126021700973_1.html/))\n\n- Industry/government context: Sarvam AI is part of India’s sovereign AI push, with Indian government programs involving multiple organizations to develop models from Indian datasets. ([business-standard.com](https://www.business-standard.com/technology/tech-news/india-ai-summit-2026-what-is-sarvam-kaze-ai-glasses-pm-modi-wore-126021700973_1.html/))\n\n- Competitive context and scope: Coverage notes competition with Meta’s Ray-Ban glasses and frames Sarvam’s emphasis on Indian-language support and on-device models as differentiators. ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/sarvam-ai-unveils-kaze-its-first-ai-powered-smart-glasses-designed-and-built-in-india/amp_articleshow/128464597.cms))'},
 {'id': 169,
  'event_id': '645_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Alibaba unveils Qwen3.5, a new agentic AI model designed to autonomously execute complex tasks (including visual agentic actions across mobile/desktop apps) and claims to outperform major U.S. rivals on several benchmarks. ([uk.finance.yahoo.com](https://uk.finance.yahoo.com/news/alibaba-group-holding-ltd-unveils-122925075.html?utm_source=openai))\n\n- Qwen3.5-397B-A17B represents 397 billion total parameters with a sparse Mixture-of-Experts design activating about 17B parameters per forward pass; the architecture includes 512 experts over 60 layers. ([marktechpost.com](https://www.marktechpost.com/2026/02/16/alibaba-qwen-team-releases-qwen3-5-397b-moe-model-with-17b-active-parameters-and-1m-token-context-for-ai-agents/?utm_source=openai))\n\n- The model is a native vision-language system trained with Early Fusion, enabling multimodal reasoning (text, images, UI screenshots, and video) and supporting 201 languages; it is designed for agentic workflows. ([marktechpost.com](https://www.marktechpost.com/2026/02/16/alibaba-qwen-team-releases-qwen3-5-397b-moe-model-with-17b-active-parameters-and-1m-token-context-for-ai-agents/?utm_source=openai))\n\n- Qwen3.5-Plus is a hosted variant offering extended context (up to 1 million tokens) via Alibaba Cloud Model Studio, while the base Qwen3.5-397B-A17B uses native 256k context. ([datacamp.com](https://www.datacamp.com/blog/qwen3-5))\n\n- Alibaba markets Qwen3.5 as “60% cheaper to use” and capable of eightfold higher throughput for large workloads versus its immediate predecessor, with claimed speedups varying by context window (e.g., 8.6x at 32K vs 256K token contexts). ([digitalapplied.com](https://www.digitalapplied.com/blog/qwen-3-5-agentic-ai-benchmarks-guide?utm_source=openai))\n\n- The company asserts Qwen3.5 outperforms leading Western models on multiple benchmarks, naming GPT-5.2, Claude Opus 4.5, and Gemini 3 Pro in its claims. ([uk.finance.yahoo.com](https://uk.finance.yahoo.com/news/alibaba-group-holding-ltd-unveils-122925075.html?utm_source=openai))\n\n- The launch is positioned within a broader push by Alibaba into AI infrastructure, including a 380-billion-yuan investment plan for 2025–2028, as it seeks to grow Qwen adoption amid domestic competition (ByteDance’s Doubao and DeepSeek). ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/tech/artificial-intelligence/alibaba-unveils-new-qwen3-5-model-for-agentic-ai-era/articleshow/128416327.cms))\n\n- The release occurred on February 16, 2026, and has been covered by Reuters and downstream outlets (e.g., Economic Times), highlighting its agentic capabilities and cost/performance claims. ([uk.finance.yahoo.com](https://uk.finance.yahoo.com/news/alibaba-group-holding-ltd-unveils-122925075.html?utm_source=openai))\n\n- Notable benchmarks and technical details (including exact scores) are surfaced in third-party writeups and developer guides, with open-weight availability for the core model and hosted options for enterprise use. ([marktechpost.com](https://www.marktechpost.com/2026/02/16/alibaba-qwen-team-releases-qwen3-5-397b-moe-model-with-17b-active-parameters-and-1m-token-context-for-ai-agents/?utm_source=openai))'},
 {'id': 170,
  'event_id': '649_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Skygen.AI declares a $7 million seed round closed and launches the world’s first autonomous Execution Layer, led by SF-based founder Mike Shperling. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))\n\n- Seed funding is provided by MLVentures (investors in Looksery and AI Factory), totaling $7 million. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/17/3238923/0/en/Mike-Shperling-s-Skygen-AI-Unveils-Multi-Tier-Security-Architecture-for-Autonomous-AI-Agents.html))\n\n- Company location and founder: San Francisco, California; founder is 19-year-old Mike Shperling. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))\n\n- The product is an autonomous software agent platform that can operate any software via a proprietary Computer Use mode, “seeing” the screen in real time and interacting with CRM, ERP, and banking interfaces 2–3x faster than existing alternatives. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))\n\n- The announcement highlights an autonomous Execution Layer as a core delivery model for scaling automation across enterprise environments. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))\n\n- Architectural design centers on a central orchestrator with Gemini Flash sub-agents to deliver high performance on long, complex tasks. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/17/3238923/0/en/Mike-Shperling-s-Skygen-AI-Unveils-Multi-Tier-Security-Architecture-for-Autonomous-AI-Agents.html))\n\n- In-context learning is a pillar: agents learn during interactions and store key insights (contacts, channels, workflows) as structured bullets to improve subsequent tasks. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))\n\n- Deep Autonomous Analysis (Deep Research) enables autonomous market analysis and data retrieval with high depth and accuracy, beyond standard AI search. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/17/3238923/0/en/Mike-Shperling-s-Skygen-AI-Unveils-Multi-Tier-Security-Architecture-for-Autonomous-AI-Agents.html))\n\n- Zero-trust security architecture: isolated virtual machines (sandboxes) with multiple security tiers, least-privilege access, and guardrails that require user permission for critical actions; user data remains within a protected perimeter and is not used for model training. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/17/3238923/0/en/Mike-Shperling-s-Skygen-AI-Unveils-Multi-Tier-Security-Architecture-for-Autonomous-AI-Agents.html))\n\n- The platform is described as designed to integrate with enterprise systems (CRM, ERP, banking interfaces) to automate digital labor at scale. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/16/3238558/0/en/Skygen-AI-Steps-Out-of-Stealth-19-Year-Old-Founder-7M-Seed-Round-and-the-End-of-the-Chatbot-Era.html))'},
 {'id': 171,
  'event_id': '650_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core development: Infosys and Anthropic announce a strategic collaboration to integrate Anthropic’s Claude models (including Claude Code) with Infosys Topaz to build agentic AI solutions for telecommunications initially, with planned expansion to financial services, manufacturing, and software development. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Center of Excellence: A dedicated Anthropic Center of Excellence will be established in telecommunications to build and deploy AI agents tailored to industry-specific operations, with expansion to other industries over time. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Agentic AI focus: The collaboration centers on agentic AI—systems that independently handle multi-step tasks—using Claude Agent SDK to create AI agents that work persistently across long, complex processes. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Industry scope: Initial focus on telecommunications, with planned expansion into financial services, manufacturing, and software development. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Claude Code integration: Claude models, including Claude Code, will be used to accelerate software development, with Infosys already deploying Claude Code in its Exponential Engineering organization. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Topaz integration: Infosys Topaz will be combined with Claude models to automate complex workflows, accelerate software delivery, and support governance/transparency in regulated industries. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- Legacy modernization: The collaboration aims to modernize legacy systems and speed up migration to AI-powered solutions across targeted industries. ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html))\n- India market context: India is a major center for Claude.ai activity, and Infosys is one of Anthropic’s early partners in Anthropic’s expanded presence in India. ([anthropic.com](https://www.anthropic.com/news/anthropic-infosys))\n- Executive perspectives: Dario Amodei (Anthropic) notes the need for domain expertise to bridge demos and regulated deployments; Salil Parekh (Infosys) emphasizes AI as a transformative enterprise capability. ([anthropic.com](https://www.anthropic.com/news/anthropic-infosys))'},
 {'id': 172,
  'event_id': '651_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core development: Alibaba releases Qwen3.5, the latest in the Qwen family, with agentic capabilities enabling autonomous, multi-step task execution. ([finance.yahoo.com](https://finance.yahoo.com/news/alibaba-unveils-qwen3-5-model-090141816.html?utm_source=openai))\n- Benchmark and cost claim: Alibaba says Qwen3.5 matches or surpasses major US models on several benchmarks and offers cost advantages, positioning it strongly in the agentic-AI race. ([finance.yahoo.com](https://finance.yahoo.com/news/alibaba-unveils-qwen3-5-model-090141816.html?utm_source=openai))\n- Open-source variant and pricing: Qwen3.5-Plus is released as an open-weight/open-source option with API pricing around 0.8 yuan per million tokens, and hosted usage supporting up to 1M token contexts. ([caixinglobal.com](https://www.caixinglobal.com/2026-02-16/alibaba-unveils-qwen35-plus-undercutting-gemini-3-pro-on-cost-102415320.html?utm_source=openai))\n- Model scale and activation: The flagship Qwen3.5 family uses 397B total parameters but activates 17B per forward pass (sparse MoE), enabling high efficiency. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Context length: Native context window is 262,144 tokens, with hosted Qwen3.5-Plus enabling up to 1,000,000 tokens. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Language support: Qwen3.5 supports 201 languages and dialects. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Agent-oriented architecture: Qwen3.5 incorporates a Model Context Protocol (MCP) and robust function-calling, designed for building AI agents. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Multimodal foundation: Native vision-language capabilities with Early Fusion training on text and image data, enabling cross-modal reasoning. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Benchmarks and capabilities: Reported scores across multiple benchmarks (e.g., AIME26, LiveCodeBench v6, MMMU, MathVision) indicate strong performance for vision, reasoning, and agentic tasks. ([nxcode.io](https://www.nxcode.io/resources/news/qwen-3-5-developer-guide-api-visual-agents-2026?utm_source=openai))\n- Market and timeline context: Announcement date February 16, 2026, with coverage highlighting Alibaba’s intensified competition in China’s chatbot/agent race and broader global AI-adoption dynamics. ([finance.yahoo.com](https://finance.yahoo.com/news/alibaba-unveils-qwen3-5-model-090141816.html?utm_source=openai))'},
 {'id': 173,
  'event_id': '652_2026-02-17',
  'news_date': '2026-02-17',
  'output': "- Moonshot targets a $10 billion valuation in an expansion of its funding round, with late January 2026 discussions, backed by Alibaba and Tencent and 5Y Capital; more than $700 million has already been committed to the round. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Moonshot is a Beijing-based AI startup behind the Kimi chatbot. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- The expansion aims to tap strong investor interest in Chinese AI model developers to rival Silicon Valley. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Backers Alibaba Group Holding Ltd. and Tencent Holdings Ltd., with 5Y Capital also participating, are part of the round. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Existing backers have committed more than $700 million to the round. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- The round is described as an expansion of the funding round rather than a brand-new financing. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Kimi is Moonshot's flagship chatbot/LLM product. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Bloomberg’s coverage of the development was published on February 17, 2026. ([news.bloomberglaw.com](https://news.bloomberglaw.com/antitrust/china-ai-startup-moonshot-seeks-10-billion-value-in-new-funding))\n- Moonshot’s Kimi K2 product is designed for complex analytical tasks such as multi-step problem solving, coding, and document-level analysis. ([thesaasnews.com](https://www.thesaasnews.com/news/moonshot-ai-raises-500m-series-c-at-4-3b-valuation?utm_source=openai))"},
 {'id': 174,
  'event_id': '653_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Adani Group announces a direct plan to invest $100 billion by 2035 to build renewable-energy–powered, hyperscale AI data centers across India, expanding AdaniConneX capacity from about 2 GW to 5 GW. ([prnewswire.com](https://www.prnewswire.com/in/news-releases/adani-group-to-invest-100-billion-in-renewable-powered-ai-data-centres-by-2035-302689648.html?utm_source=openai))\n\n- The plan is expected to catalyze roughly $150 billion more in related investments (servers, cloud platforms, electrical infrastructure), creating a projected $250 billion AI infrastructure ecosystem in India by 2035. ([prnewswire.com](https://www.prnewswire.com/in/news-releases/adani-group-to-invest-100-billion-in-renewable-powered-ai-data-centres-by-2035-302689648.html?utm_source=openai))\n\n- AdaniConneX expansion from 2 GW to 5 GW will occur across multiple Indian campuses, forming what the company terms a large, integrated energy-and-compute platform. ([prnewswire.com](https://www.prnewswire.com/in/news-releases/adani-group-to-invest-100-billion-in-renewable-powered-ai-data-centres-by-2035-302689648.html?utm_source=openai))\n\n- Google is partnering with Adani to develop a gigawatt-scale AI data center campus in Visakhapatnam; separately, Adani is working with Microsoft on AI facilities in Hyderabad and Pune. ([prnewswire.com](https://www.prnewswire.com/in/news-releases/adani-group-to-invest-100-billion-in-renewable-powered-ai-data-centres-by-2035-302689648.html?utm_source=openai))\n\n- The data centers are to be powered by a renewable-energy backbone anchored in Adani Green Energy’s Khavda project (about 30 GW), with more than 10 GW already operational. ([prnewswire.com](https://www.prnewswire.com/in/news-releases/adani-group-to-invest-100-billion-in-renewable-powered-ai-data-centres-by-2035-302689648.html?utm_source=openai))\n\n- The initiative includes expanding a data-center collaboration with Flipkart to support AI workloads and broader e‑commerce AI needs, aligning hyperscale capacity with retail operations. ([techcrunch.com](https://techcrunch.com/2026/02/17/adani-pledges-100b-for-ai-data-centers-as-india-seeks-bigger-role-in-global-ai/?utm_source=openai))\n\n- The program envisions a five-layer AI stack centered on India’s technological sovereignty, effectively tying energy generation, transmission, and high-density AI compute into a single platform. ([indiatoday.in](https://www.indiatoday.in/amp/business/story/adani-group-announces-100-billion-to-build-ai-ready-green-data-centres-by-2035-2869553-2026-02-17?utm_source=openai))\n\n- Adani’s announcement highlights India’s aim to position the country as a global supplier of AI infrastructure, branding the effort as one of the world’s largest integrated energy–compute commitments. ([business-standard.com](https://www.business-standard.com/markets/capital-market-news/adani-enterprises-rises-on-100-bn-sovereign-ai-infrastructure-investment-plan-126021700511_1.html?utm_source=openai))\n\n- Context: the move comes amid broader Indian efforts to attract substantial data-center investment (potentially up to about $200 billion) and to support AI hub ambitions, with major players like Google, Microsoft, and others expanding in the region. ([apnews.com](https://apnews.com/article/4e170da0a3b883a9659569dd538e9019?utm_source=openai))'},
 {'id': 175,
  'event_id': '654_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Pershing Square disclosed a roughly $2 billion stake in Meta Platforms, representing about 10% of the fund’s capital, as part of its AI-directed investment bet. ([channelnewsasia.com](https://www.channelnewsasia.com/business/ackmans-hedge-fund-pershing-square-bets-meta-exits-hilton-5924456))\n- The disclosure occurred on February 11, 2026, during Pershing Square’s annual investor presentation to clients. ([channelnewsasia.com](https://www.channelnewsasia.com/business/ackmans-hedge-fund-pershing-square-bets-meta-exits-hilton-5924456))\n- The stake amounts to about 10% of Pershing Square’s capital and roughly $2 billion in value. ([channelnewsasia.com](https://www.channelnewsasia.com/business/ackmans-hedge-fund-pershing-square-bets-meta-exits-hilton-5924456))\n- The investment thesis argues Meta’s AI integration could yield meaningful long-term upside, with the team stating the current share price underestimates AI-driven potential. ([channelnewsasia.com](https://www.channelnewsasia.com/business/ackmans-hedge-fund-pershing-square-bets-meta-exits-hilton-5924456))\n- Meta’s 2026 AI-related expenditures are forecast to total between $115 billion and $135 billion, highlighting the scale of its AI ambitions. ([foxbusiness.com](https://www.foxbusiness.com/markets/bill-ackman-makes-2b-gamble-mark-zuckerbergs-ai-pivot-massive-meta-stock-purchase?utm_source=openai))\n- Meta’s stock has traded lower amid AI-spending concerns, with notable declines in the months leading into early 2026. ([wsj.com](https://www.wsj.com/finance/investing/bill-ackman-makes-a-big-bet-on-meta-e4f4b2cf?utm_source=openai))\n- The stake sits within Pershing Square’s concentrated portfolio and aligns with Ackman’s broader AI tilt, which includes positions in Alphabet and Amazon. ([barrons.com](https://www.barrons.com/articles/pershing-ackman-stakes-meta-amazon-hertz-e59ba896?utm_source=openai))\n- The Meta stake was initiated in November 2025, at roughly $625 per share, according to the investor presentation. ([wsj.com](https://www.wsj.com/finance/investing/bill-ackman-makes-a-big-bet-on-meta-e4f4b2cf?utm_source=openai))'},
 {'id': 176,
  'event_id': '658_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core development: Cohere launched Tiny Aya, a family of open-weight multilingual models (base ~3.35B parameters) designed for offline/on-device use, spanning 70+ languages; announced at the India AI Summit. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Open-weight and accessibility: Tiny Aya is an open-weights research release with public access to the weights and tooling; aims for on-device testing and research use. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Regional variants: The family includes Tiny Aya Earth (West Asian and African languages), Tiny Aya Fire (South Asian languages), Tiny Aya Water (Asia Pacific/West Asia/Europe), plus a broader Tiny Aya Global variant for wider coverage. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Base model details: The base model contains 3.35 billion parameters and supports 70+ languages. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Language coverage: Includes languages such as Bengali, Hindi, Punjabi, Urdu, Gujarati, Tamil, Telugu, and Marathi, among others. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- On-device/offline capability: Designed to run directly on devices without requiring an internet connection, enabling offline translation and use. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Training hardware: Models were trained on a single cluster of 64 Nvidia H100 GPUs. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n- Context length: Tiny Aya models support an 8K token input context length (and 8K output length for generation). ([huggingface.co](https://huggingface.co/CohereLabs/tiny-aya-earth))\n- Availability and platforms: Weights and tooling are available on Hugging Face, with distribution also noted for Kaggle and Ollama for local deployment. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))'},
 {'id': 177,
  'event_id': '660_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core new development: AlphaFold Protein Database now serves over 3 million researchers across more than 190 countries and provides free access to about 240 million predicted protein structures. ([fortune.com](https://fortune.com/2026/02/16/google-deepmind-ceo-demis-hassabis-james-manyika-transforming-sciecne-alphafold/))\n\n- Scale and openness context: The AFDB is freely accessible to the global research community, with collaboration between Google DeepMind and EMBL-EBI expanding coverage to hundreds of millions of protein predictions. ([alphafold.ebi.ac.uk](https://alphafold.ebi.ac.uk/about?utm_source=openai))\n\n- Latest platform expansion: In 2024, AlphaFold 3 and the AlphaFold Server were launched, with the server offering free access to AF3 for non-commercial research and enabling protein interactions predictions. ([deepmind.google](https://deepmind.google/science/alphafold/?utm_source=openai))\n\n- Regional impact examples: Use cases highlighted include researchers in Malaysia studying Melioidosis and teams in India breeding charcoal-rot resistant soybeans, illustrating regional problems addressed at global scale. ([fortune.com](https://fortune.com/2026/02/16/google-deepmind-ceo-demis-hassabis-james-manyika-transforming-sciecne-alphafold/))\n\n- Historical scale growth (contextual baseline): The AFDB previously surpassed 200 million predicted structures, with rapid expansion since 2021; ongoing updates continue to broaden coverage. ([embl.org](https://www.embl.org/news/science/alphafold-200-million/?utm_source=openai))\n\n- Nobel Prize context: The AlphaFold advancement contributed to the 2024 Nobel Prize in Chemistry awarded to Demis Hassabis and John Jumper for protein structure prediction; this milestone underpins the database’s prominence. ([theguardian.com](https://www.theguardian.com/science/2024/oct/09/google-deepmind-scientists-win-nobel-chemistry-prize?utm_source=openai))\n\n- Isoforms and UniProt integration: The 2025 AFDB update synchronized with UniProt to include protein isoforms and to improve proteome coverage; MSAs for each prediction are now available for download. ([embl.org](https://www.embl.org/news/science-technology/google-deepmind-partnership-renewal/?utm_source=openai))\n\n- Data size reference: The AlphaFold Database data footprint is substantial (reported around 23 terabytes in recent summaries), reflecting the scale of open-access structural data. ([quantumrun.com](https://www.quantumrun.com/consulting/alphafold-2-statistics/?utm_source=openai))\n\n- Disease relevance signal: More than 30% of papers citing AlphaFold relate to disease research, underscoring its impact on biomedical discovery. ([deepmind.google](https://deepmind.google/science/alphafold/?utm_source=openai))'},
 {'id': 178,
  'event_id': '661_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- SurrealDB 3.0 launches alongside a $23 million Series A extension, bringing total funding to $44 million and introducing a single Rust-native engine that unifies relational, document, graph, vector, and time-series data to power agentic AI workflows. ([venturebeat.com](https://venturebeat.com/data/surrealdb-3-0-wants-to-replace-your-five-database-rag-stack-with-one))\n\n- SurrealDB 3.0 embeds agent memory and business logic inside the database via the Surrealism plugin system, enabling transactional memory and in-database logic for AI agents. ([venturebeat.com](https://venturebeat.com/data/surrealdb-3-0-wants-to-replace-your-five-database-rag-stack-with-one))\n\n- SurrealDB reports 2.3 million downloads and about 31,000 GitHub stars to date, signaling broad developer traction alongside the platform update. ([venturebeat.com](https://venturebeat.com/data/surrealdb-3-0-wants-to-replace-your-five-database-rag-stack-with-one))\n\n- Sphinx raises a $7.1 million seed round led by Cherry Ventures, with participation from Y Combinator, Rebel Fund, Deel Ventures, and Singularity Capital, to scale its browser-native AI compliance agents. ([venturebeat.com](https://venturebeat.com/business/sphinx-raises-71m-to-build-every-financial-institutions-last-compliance-hire))\n\n- Sphinx’s agents run directly inside existing enterprise tools (case management systems, PDFs, emails, dashboards) with no required replacements, performing AML/KYC/KYB tasks and producing regulator-ready audit trails. ([venturebeat.com](https://venturebeat.com/business/sphinx-raises-71m-to-build-every-financial-institutions-last-compliance-hire))\n\n- Cohere launches Tiny Aya, a family of open multilingual models (base 3.35B parameters) that support 70+ languages and run offline on consumer devices, with variants tailored for regions and on-device use; models are available on HuggingFace, Kaggle, and Ollama. ([techcrunch.com](https://techcrunch.com/2026/02/17/cohere-launches-a-family-of-open-multilingual-models/))\n\n- Cohere is also reported to have posted $240 million in annual recurring revenue for 2025 and is signaling an IPO in the near term. ([techcrunch.com](https://techcrunch.com/2026/02/13/coheres-240m-year-sets-stage-for-ipo/))\n\n- Infosys and Anthropic announce a strategic collaboration to bring Claude models (including Claude Code) to Infosys Topaz AI, enabling agentic AI across regulated industries with an Anthropic Center of Excellence and internal deployment to accelerate client work. ([techcrunch.com](https://techcrunch.com/2026/02/17/as-ai-jitters-rattle-it-stocks-infosys-partners-with-anthropic-to-build-enterprise-grade-ai-agents/)) ([infosys.com](https://www.infosys.com/newsroom/press-releases/2026/advanced-enterprise-ai-solutions-industries.html?utm_source=openai))\n\n- India aims to attract over $200 billion in AI infrastructure investment by 2028, backed by policy levers (tax relief, a government-backed venture program) and an expansion of shared compute capacity, including a plan to add thousands of GPUs. ([techcrunch.com](https://techcrunch.com/2026/02/17/india-bids-to-attract-over-200b-in-ai-infrastructure-investment-by-2028/))\n\n- The India AI Impact Summit in New Delhi features participation from OpenAI, Anthropic, Google, and other global firms, with government officials outlining aggressive compute-infrastructure goals and related policy support. ([techcrunch.com](https://techcrunch.com/2026/02/17/india-bids-to-attract-over-200b-in-ai-infrastructure-investment-by-2028/))'},
 {'id': 179,
  'event_id': '665_2026-02-17',
  'news_date': '2026-02-17',
  'output': '- Core development: Figma and Anthropic announce Code to Canvas, a Claude-based workflow to turn AI-generated code into fully editable Figma designs, enabling code-to-design import and interactive prototypes. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/tech/artificial-intelligence/figma-partners-with-anthropic-to-launch-code-to-canvas/articleshow/128475451.cms?from=mdr))\n\n- Announcement date and framing: The feature was unveiled on February 17, 2026, as part of a broader collaboration tying Claude Code to the Figma design platform. ([figma.com](https://www.figma.com/blog/the-future-of-design-is-code-and-canvas/?utm_source=openai))\n\n- How it works (mechanism): Users install the Figma Model Context Protocol (MCP) server and can send Claude Code outputs to Figma, which translate into editable layers on the canvas. ([figma.com](https://www.figma.com/blog/the-future-of-design-is-code-and-canvas/?utm_source=openai))\n\n- FigJam support: Claude can generate editable FigJam diagrams from prompts, PDFs, images, or screenshots, bringing AI-driven diagramming into collaborative workflows. ([figma.com](https://www.figma.com/blog/think-outside-of-the-box-with-claude-and-figjam/?utm_source=openai))\n\n- Claude Apps integration: Anthropic added interactive Apps (including Figma) to Claude’s interface, enabling workplace tools to run inside Claude (pro/enterprise tiers; not available to free users). ([techcrunch.com](https://techcrunch.com/2026/01/26/anthropic-launches-interactive-claude-apps-including-slack-and-other-workplace-tools/))\n\n- Claude Code plugin for Figma: Anthropic’s Claude Code can connect directly to Figma files via a dedicated plugin, enabling extraction of design data (layouts, tokens), mapping to code with Code Connect, and rendering production-ready code. ([claude.com](https://claude.com/plugins/figma))\n\n- Roundtrip workflow: The Claude Code to Figma capture (and back) flow is framed as a “roundtrip” enabled by the MCP server, allowing design artifacts to seed code generation and vice versa. ([figma.com](https://www.figma.com/blog/introducing-claude-code-to-figma/?utm_source=openai))\n\n- Context within Figma’s AI push: The announcement sits alongside Figma’s broader AI roadmap (Make for prompt-to-code and related canvas experiments), illustrating deeper integrations of code, design, and AI workflows. ([figma.com](https://www.figma.com/blog/the-future-of-design-is-code-and-canvas/?utm_source=openai))\n\n- External coverage confirming scope: Coverage highlights include opportunities to generate diagrams in FigJam via Claude and the broader Claude apps integration, underscoring a product-level AI feature partnership between a leading design platform and an LLM provider. ([cmswire.com](https://www.cmswire.com/digital-experience/figma-integrates-anthropics-claude-to-generate-diagrams-inside-figjam/?utm_source=openai))'}]

In [52]:
claude_article_builder(a)

{'headline': "Meta's $2B Manus AI Now Automates Your Ad Campaigns", 'output': "Meta has started rolling out Manus AI inside Ads Manager, letting advertisers automate grunt work like data analysis and report generation. Manus, acquired in December for over $2 billion, is an autonomous AI agent that handles multi-step tasks without constant human input. Think of it as a smart intern that actually delivers: it can build reports, research audiences, and even code. Meta severed all Chinese ties post-acquisition, relocating Manus to Singapore. This marks Meta's pivot from chatbots to agentic AI, agents that execute complex workflows independently. As Manus scales across Meta's platforms, advertisers could spend less time on spreadsheets and more time on strategy.", 'sources': [{'name': 'Social Media Today', 'url': 'https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/?utm_source=openai'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2025/12/

In [24]:
def save_article(article_json):
    supabase.table('hundred_word_articles').insert({
        'event_id': article_json['event_id'],
        'model_provider': article_json['model_provider'],
        'news_date': article_json['news_date'],
        'sources': json.dumps(article_json['sources']),
        'output': article_json['output'],
        'headline': article_json['headline']
    }).execute()

In [53]:
final_summary_4

[{'headline': 'India Hosts First Global South AI Summit, Tech Titans Descend',
  'output': "India is making history as the first Global South nation to host a major AI summit. The India AI Impact Summit 2026 kicks off February 16-20 in New Delhi, with Prime Minister Modi inaugurating the expo. Expect 20 heads of state and 40+ tech CEOs, including Google's Sundar Pichai and Anthropic's Dario Amodei (Sam Altman's attendance remains unconfirmed). The summit focuses on moving AI from pilot projects to real-world policy and action, organized around three themes: People, Planet, and Progress. This positions India as a key player in shaping inclusive AI governance globally, potentially unlocking massive partnerships and investment for emerging markets.",
  'sources': [{'name': 'India AI Impact Summit Official Site',
    'url': 'https://www.impactexpo.indiaai.gov.in/?utm_source=openai'},
   {'name': 'Times of India',
    'url': 'https://timesofindia.indiatimes.com/technology/tech-news/india-ai

In [57]:
for i in final_summary_4:
    if i['news_date'] >= '2026-02-17':
        save_article(i)
        print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [25]:
for i in final_summary_3:
    save_article(i)
    print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [249]:

fail = []
pas = 0

for i in final_summary_3:
    words = i['output'].split()
    word_count = len(words)
    if word_count > 100:
        print(f"Fail: {word_count}")
        fail.append(word_count)
    else:
        print(f"Pass: {word_count}")
        pas +=1

print(f"Fail: {sum(fail)/len(fail)}")



Pass: 92
Pass: 93
Pass: 98
Pass: 95
Pass: 100
Pass: 99
Pass: 87
Pass: 91
Pass: 92
Pass: 97
Pass: 97
Pass: 100
Fail: 103
Pass: 92
Pass: 89
Pass: 92
Pass: 86
Fail: 101
Pass: 94
Pass: 89
Fail: 101
Fail: 110
Pass: 96
Fail: 105
Pass: 94
Fail: 102
Fail: 101
Pass: 83
Fail: 103.28571428571429


In [223]:
import statistics as stats